In [35]:
import polynomial_matching as pm
import cv2
import numpy as np
import matplotlib.pyplot as plt
import rawpy
import pandas as pd

## Read images

In [36]:
raw_images = []
for i in range(1,6):
    raw = cv2.imread('data/image/Canon1DsMkIII_000{}.tiff'.format(i), cv2.IMREAD_UNCHANGED)
    raw_images.append(cv2.demosaicing(raw, cv2.COLOR_BAYER_BG2BGR))

In [37]:
def normIm(image, darkness, saturation):
    '''Normalize the image to [0,1]'''
    return (image-darkness)/(saturation-darkness)

In [38]:
darkness_level = 1024.
saturation_level = 15279.

In [39]:
norm_images = []
for im in raw_images:
    norm_images.append(normIm(im, 1024., 15279))

In [40]:
def brightCorrection(image):
    '''Adjust the brightness of the image'''
    # RGB转灰度 RGB 0.299, 0.587, 0.114
    grayMean = np.mean(image[:,:,0])*0.114 + np.mean(image[:,:,1])*0.587 + np.mean(image[:,:,2])*0.299
    grayScale = 0.25/grayMean
    rtn = image*grayScale
    rtn[rtn>1] = 1
    return rtn

In [41]:
def gammaCorrection(image, gamma=2.2):
    '''Convert linear RGB to sRGB with gammaCorrection'''
    invgamma = 1 / gamma
    return (np.power(image, invgamma)*255).astype(np.uint8)

In [42]:
sRGB_images = []
for im in norm_images:
    print(np.shape(im))
    sRGB_images.append(cv2.cvtColor(gammaCorrection(brightCorrection(im)),cv2.COLOR_BGR2RGB))

(3752, 5640, 3)


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1, 5)

ax[0].imshow(sRGB_images[0])
ax[1].imshow(sRGB_images[1])
ax[2].imshow(sRGB_images[2])
ax[3].imshow(sRGB_images[3])
ax[4].imshow(sRGB_images[4])
plt.show()

## Read patch color

In [ ]:
patches = []
for i in range(1, 6):
    rawpatch = pd.read_csv('data/checker/Canon1DsMkIII_000{}_color.txt'.format(i), names=['1','2','3']).to_numpy() # access to the RAW image
    patches.append(gammaCorrection(normIm(rawpatch,darkness_level,saturation_level)))
    
print(patches[0])

In [ ]:
print(patches[0][:,0])

In [ ]:
stdmatch = []
stdimages = []
for i in range(0, len(patches)):
    stdmatch.append(pm.polyCoeff(patches[i], pm.ColorChecker2005_sRGB))
    stdmatch[-1].to_npy('data/Canon1DsMkIII_000{}.npy'.format(i))
    stdimages.append(stdmatch[-1].transform(sRGB_images[i]))

In [ ]:
fig, ax = plt.subplots(1, 5)

ax[0].imshow(stdimages[0])
ax[1].imshow(stdimages[1])
ax[2].imshow(stdimages[2])
ax[3].imshow(stdimages[3])
ax[4].imshow(stdimages[4])
plt.show()

In [ ]:
cv2.imwrite('error_image.png', cv2.cvtColor(stdimages[4], cv2.COLOR_RGB2BGR))

In [ ]:
plt.imshow(cv2.cvtColor(sRGB_images[4],cv2.COLOR_BGR2RGB))

In [ ]:
uniquematch=[]
uniqueimages=[]
for i in range(0, len(patches)-1):
    uniquematch.append(pm.polyCoeff(pm.ColorChecker2005_sRGB, patches[i]))
    uniquematch[-1].to_npy('data/Canon1DsMkIII_000{}.npy'.format(i))
    uniqueimages.append(uniquematch[-1].transform(stdimages[0]))

In [ ]:
fig, ax = plt.subplots(2, 2)

ax[0, 0].imshow(uniqueimages[0])
ax[0, 1].imshow(uniqueimages[1])
ax[1, 0].imshow(uniqueimages[2])
ax[1, 1].imshow(uniqueimages[3])
plt.show()